In [17]:

#from tkinter import *
#import customtkinter

In [2]:
import tkinter as tk
import subprocess
import os
import time
from tkinter import ttk, messagebox
class MyApplication(tk.Tk):
    def __init__(self):

        super().__init__()
        self.title("Or-Q")
        self.geometry("600x400")

        # -------------------------------------------- STYLE ---------------------------------------------
        
        style = ttk.Style(self)
        style.theme_use("alt")
        style.configure("TButton", font=("Montserrat", 8), padding=5)
        style.configure("Red.TButton", background="red", foreground="black", font=("Montserrat", 8))
        style.configure("Green.TButton", background="green", foreground="black", font=("Montserrat", 8))

        style.configure("TLabel", font=("Montserrat", 8), foreground="black", background="#f0f0f0")
        style.configure("Red.TLabel", foreground="red", font=("Montserrat", 8), background="#f0f0f0")
        style.configure("Green.TLabel",foreground="green", font=("Montserrat", 8), background="#f0f0f0")
        style.configure("Grey.TLabel",foreground="grey", font=("Montserrat", 8), background="#f0f0f0")
                
        style.map("TButton",background=[("active", "lightgrey"), ("pressed", "lightgrey")],
          foreground=[("active", "grey"), ("pressed", "grey")])
        
        style.map("Green.TButton",background=[("active", "lightgreen"), ("pressed", "lightgreen")],
          foreground=[("active", "green"), ("pressed", "green")])

        style.map("Red.TButton",background=[("active", "pink"), ("pressed", "pink")],
          foreground=[("active", "red"), ("pressed", "red")])
        
        # -------------------------------------------- DEFINE VARIABLES ---------------------------------------------
        
            # Boolean variables
        self.omni_running = tk.BooleanVar(value = False) # status of OMNI: running?
        self.imed_running = tk.BooleanVar(value = False) # status of IMED: running?
        self.variables_set = tk.BooleanVar(value = False) # required datapoints are defined?
        self.process_stop = tk.BooleanVar(value = False) # required datapoints are defined?
        
            # Integer variables
        self.omni_run_count = tk.IntVar(value = 0.0) # count the number of omni runs
        self.imed_run_count = tk.IntVar(value = 0.0) # count the number of IMED runs
        self.initial_exp_count = tk.IntVar(value = 0.0)

            # String variables
        self.status_message = tk.StringVar(value='') 
        self.error_message = tk.StringVar(value='') 
        self.variables_set_message = tk.StringVar(value='')
        
        self.omni_text = tk.StringVar(value = "OMNIBUS")
        self.imed_text = tk.StringVar(value = "IMED")
        self.experiment_folder = tk.StringVar(value = "")
        self.imed_folder = tk.StringVar(value = "")
        
        self.omni_filename = 'expttsd'
        self.imed_filename = 'imed'

        # --------------------------- Create Widgets ----------------------------------
        self.create_labels()
        self.create_buttons()
        self.create_textboxes()
        self.layout_widgets()
        
    # Labels
    def create_labels(self):
        
        self.Omni_label1 = ttk.Label(self,  textvariable=self.omni_text)
        self.Imed_label1 = ttk.Label(self,  textvariable=self.imed_text)
        self.general_label1 = ttk.Label(self, text = 'Currently running:')
        self.general_label2 = ttk.Label(self, text = 'No. of runs:')
        self.general_label3 = ttk.Label(self, textvariable = self.status_message)
        self.general_label4 = ttk.Label(self, textvariable = self.error_message)
        self.general_label5 = ttk.Label(self, textvariable = self.variables_set_message)
        self.Omni_label2 = ttk.Label(self, text=str(self.omni_run_count.get()))
        self.Imed_label2 = ttk.Label(self, text=str(self.imed_run_count.get()))
        self.text_exp_loc_label1 = ttk.Label(self, text = 'Exp. folder path') 
        self.text_imed_loc_label1 = ttk.Label(self, text = 'IMED. folder path') 
        self.text_no_exp_label1 = ttk.Label(self, text = 'No. of exp:')
        
    # Buttons
    def create_buttons(self):
    
        self.start_button  = ttk.Button(self, text="Start automated experiments", command=self.start_command)
        self.define_button = ttk.Button(self, text="Define settings", command=self.define_command)
        self.quit_button   = ttk.Button(self, text="Quit", width=8, command=self.destroy)         
        self.stop_button   = ttk.Button(self, text="Stop", width=8, command=self.stop_command)       
        self.reset_button  = ttk.Button(self, text="Reset", command=self.reset_command)
    
    # Text boxes
    def create_textboxes(self):
        
        self.text_no_exp = tk.Text(self, height=1, width=5) # Text box
        self.text_exp_loc = tk.Text(self, height=1, width=20) # Text box
        self.text_imed_loc = tk.Text(self, height=1, width=20) # Text box
    
    # ------------------------------------- PLACE VARIABLES ---------------------------------------------

    def layout_widgets(self):
        
        self.Omni_label1.place(x=400,y=210)
        self.Imed_label1.place(x=500,y=210)
        self.general_label1.place(x=400,y=180)
        self.general_label2.place(x=400,y=240)
        self.Omni_label2.place(x=400,y=270)
        self.Imed_label2.place(x=500,y=270)
        self.general_label3.place(x=150,y=70)
        self.general_label4.place(x=150,y=90)
        self.general_label5.place(x=150,y=180)
        self.text_exp_loc_label1.place(x=50,y=120)
        self.text_imed_loc_label1.place(x=50,y=160)
        self.text_no_exp_label1.place(x=70,y=140)

        self.start_button.place(x=115,y=270)
        self.define_button.place(x=150,y=200)
        self.quit_button.place(x=30,y=270)
        self.stop_button.place(x=300,y=270)
        self.reset_button.place(x=300,y= 310)

        self.text_no_exp.place(x=150,y=140)
        self.text_exp_loc.place(x=150,y=120)
        self.text_imed_loc.place(x=150,y=160)
        
    # ----------------------------------- Commands -------------------------------------------
    def check_if_empty_text(self):
        return [len(i) != 0 for i in [self.text_exp_loc.get("1.0", tk.END).strip(), self.text_imed_loc.get("1.0", tk.END).strip(), self.text_no_exp.get("1.0", tk.END).strip()]]

    def disable_buttons(self):
        for btn in (self.start_button, self.define_button, self.quit_button, self.reset_button):
            btn.state(["disabled"])

    def enable_buttons(self):
        for btn in (self.start_button, self.define_button, self.quit_button, self.reset_button):
            btn.state(["!disabled"])
            
    def define_command(self):    
        
        self.status_message.set('')
        self.error_message.set('')
        self.variables_set_message.set('')
        self.variables_set.set(False)
        
        folder_loc = self.text_exp_loc.get("1.0", tk.END).strip() # Text box
        folder_imed_loc = self.text_imed_loc.get("1.0", tk.END).strip() # Text box
        no_exp     = self.text_no_exp.get("1.0", tk.END).strip() # Text box

        if not all( self.check_if_empty_text() ):
            
            self.status_message.set("Please dont leave any brackets empty")
            self.general_label3.config(style="Red.TLabel")
            self.define_button.config(style="Red.TButton")
            
        else:
            if all([os.path.isdir(folder_loc), os.path.isdir(folder_imed_loc)] ):
                
                self.experiment_folder.set(folder_loc) 
                self.imed_folder.set(folder_imed_loc) 
                
                try:
                    no_exp_1 = int(no_exp)
                    if no_exp_1 <= 0:
                        self.error_message.set("Enter a positive integer")
                        self.general_label4.config(style="Red.TLabel")
                        self.define_button.config(style="Red.TButton")
                    else:
                        # if everything checks out:
                        
                        self.initial_exp_count.set(int(no_exp)) 
                        self.define_button.config(style="Green.TButton")
                        self.variables_set_message.set('Variables set')
                        self.general_label5.config(style="Green.TLabel")
                        self.variables_set.set(True)
                        
                except ValueError:
                    self.error_message.set("Enter a valid integer")
                    self.define_button.config(style="Red.TButton")
            else:
                self.status_message.set("Enter a valid folder path")
                self.define_button.config(style="Red.TButton")  
                
    def files_in_folder(self,V):
        
        # count the number of files in a folder with a specific file name (self.<omni or imed>_filename)
        # V: 'O' or 'I' --> determines whether its OMNI or IMED stage
        # ------------------------------------------------------------
        
        count = 0
        if V == 'O':
            folder = self.experiment_folder.get()
            filename = self.omni_filename
        elif V == 'I':
            folder = self.imed_folder.get()
            filename = self.imed_filename
        else: 
            messagebox.showerror("Error", 'Folder checker error')
            
        with os.scandir(folder) as entries:
            for entry in entries:
                if filename in entry.name:
                    count += 1
        return count
        
    
    
    def monitoring_folders(self, check_frequency,V):

        # (1.1) start monitoring the experiment folder in the automated sequence
        # (1.2) setting the ongoing process label green
        # (1.3) monitoring omnifolder until number of files equal self.initial_exp_count
        # (2.1) start monitoring imed folder once omnibus finished
        # (2.2) setting the ongoing process label green
        # (1.3) monitoring omnifolder until number of files equal self.initial_exp_count
        #                       --> should be changed to < if equals '1' >
        
        # check_frequency: checking the folder in every <check_frequency> seconds
        # V: 'O' or 'I' --> determines whether its OMNI or IMED stage
        # ------------------------------------------------------------
        
        
        file_count = self.files_in_folder(V)
        print(f'The number of files in folder: {file_count:.0f}')
        
        if file_count < self.initial_exp_count.get():
            if V == 'O':
                self.Omni_label1.configure(style="Green.TLabel")
                self.Imed_label1.configure(style="Grey.TLabel")
            elif V == 'I':
                self.Omni_label1.configure(style="Grey.TLabel")
                self.Imed_label1.configure(style="Green.TLabel")
                
            self.after(check_frequency * 1000, lambda: self.monitoring_folders(check_frequency,V))
            return False
            
        else:
            self.enable_buttons()
            if V == 'O':
                messagebox.showerror("Error", 'Omni finished')
                
                # start IMED
                self.omni_running.set(False)
                self.Omni_label1.configure(style="Grey.TLabel")
                self.imed_running.set(True)
                self.Imed_label1.configure(style="Green.TLabel")
                self.omni_run_count.set(self.omni_run_count.get() + 1)
                self.Omni_label2.config(text=str(self.omni_run_count.get()))
                self.disable_buttons()

                # if OMNIBUS complete, start IMED
                self.monitoring_folders(5, 'I')
                
            elif V == 'I':
                # 
                messagebox.showerror("Error", 'IMED finished')
                
                self.omni_running.set(False)
                self.Omni_label1.configure(style="Grey.TLabel")
                self.imed_running.set(False)
                self.Imed_label1.configure(style="Grey.TLabel")
                self.imed_run_count.set(self.imed_run_count.get() + 1)
                self.Imed_label2.config(text=str(self.omni_run_count.get()))
                
            return True
                    
    def start_command(self):

        # start the automated OMNIBUS-->IMED loops
        # ----------------------------------------
        
        print(self.files_in_folder('I'))
        if not self.variables_set.get():
            messagebox.showerror("Error", 'Please define variables first!')
        elif self.files_in_folder('O') != 0: # check stuffs to be true          
                messagebox.showerror("Error", 'Files \n already in experiment folder')
            
        elif self.files_in_folder('I') != 0: # check stuffs to be true          
                messagebox.showerror("Error", 'Files \n already in IMED folder')
        else:

            # start OMNIBUS
            self.omni_running.set(True)
            self.disable_buttons()

            # wait for results
            self.monitoring_folders(5, 'O')
               
    def stop_command(self):
        # kill the processes (still in progress to complete)
        # -----------------------------------
        
        self.process_stop.set(False)
        if not self.variables_set.get():
            messagebox.showerror("Error", 'Please define variables first!')
        else:
            self.process_stop.set(True)
            
            # rest of code:
            # taskkill omnibus
            # stop IMED in Matlab

    def reset_command(self):

        # reset all vairables and widgets
        # --------------------------------
        
        self.omni_running = tk.BooleanVar(value = False) 
        self.imed_running = tk.BooleanVar(value = False) 
        self.variables_set = tk.BooleanVar(value = False) 
        self.process_stop = tk.BooleanVar(value = False) 
        self.omni_run_count = tk.IntVar(value = 0.0) 
        self.imed_run_count = tk.IntVar(value = 0.0)
        self.initial_exp_count = tk.IntVar(value = 0.0)
        self.status_message = tk.StringVar(value='') 
        self.error_message = tk.StringVar(value='') 
        self.variables_set_message = tk.StringVar(value='')
        self.omni_text = tk.StringVar(value = "OMNIBUS")
        self.imed_text = tk.StringVar(value = "IMED")
        self.experiment_folder = tk.StringVar(value = "")
        self.imed_folder = tk.StringVar(value = "")
        self.omni_filename = 'expttsd'
        self.imed_filename = 'imed'
        
        self.create_labels()
        self.create_buttons()
        self.create_textboxes()
        self.layout_widgets()

                

if __name__ == "__main__":
    app = MyApplication()
    app.mainloop()
    

0
The number of files in folder: 0
The number of files in folder: 1
The number of files in folder: 1


In [3]:
def monitoring_folders(self, check_frequency, stage):
    """
    Monitor OMNI or IMED folders until expected files appear.
    Automatically switches stages and loops.
    """
    file_count = self.files_in_folder(stage)
    print(f'[{stage}] Files in folder: {file_count}')

    # Update labels
    if stage == 'O':
        self.Omni_label1.configure(style="Green.TLabel" if file_count < self.initial_exp_count.get() else "Grey.TLabel")
        self.Imed_label1.configure(style="Grey.TLabel")
    else:  # stage == 'I'
        self.Omni_label1.configure(style="Grey.TLabel")
        self.Imed_label1.configure(style="Green.TLabel" if file_count < self.initial_exp_count.get() else "Grey.TLabel")

    # If folder not ready, check again after delay
    if file_count < self.initial_exp_count.get():
        self.after(check_frequency * 1000, lambda: self.monitoring_folders(check_frequency, stage))
        return

    # Stage finished
    self.enable_buttons()
    if stage == 'O':
        self.omni_running.set(False)
        self.imed_running.set(True)
        self.omni_run_count.set(self.omni_run_count.get() + 1)
        self.Omni_label2.config(text=str(self.omni_run_count.get()))
        self.Imed_label1.configure(style="Green.TLabel")
        
        # Start IMED stage
        self.monitoring_folders(check_frequency, 'I')

    elif stage == 'I':
        self.imed_running.set(False)
        self.imed_run_count.set(self.imed_run_count.get() + 1)
        self.Imed_label2.config(text=str(self.imed_run_count.get()))
        self.Omni_label1.configure(style="Grey.TLabel")
        self.Imed_label1.configure(style="Grey.TLabel")

        # Finished one full loop
        self.current_run += 1
        if self.current_run < self.total_runs:
            print(f"Starting next OMNI→IMED loop ({self.current_run + 1}/{self.total_runs})")
            self.omni_running.set(True)
            self.disable_buttons()
            self.monitoring_folders(check_frequency, 'O')
        else:
            print("All loops completed!")


In [ ]:
def start_command(self):
    # Ensure variables are set and folders are empty
    if not self.variables_set.get():
        messagebox.showerror("Error", "Please define variables first!")
        return
    elif self.files_in_folder('O') != 0:
        messagebox.showerror("Error", "Files already in OMNI folder")
        return
    elif self.files_in_folder('I') != 0:
        messagebox.showerror("Error", "Files already in IMED folder")
        return

    # Initialize counters
    self.current_run = 0
    self.omni_run_count.set(0)
    self.imed_run_count.set(0)

    # Start the first OMNI stage
    self.omni_running.set(True)
    self.disable_buttons()
    self.monitoring_folders(5, 'O')


In [ ]:
def start_command(self):
    # Ensure variables are set and folders are empty
    if not self.variables_set.get():
        messagebox.showerror("Error", "Please define variables first!")
        return
    elif self.files_in_folder('O') != 0:
        messagebox.showerror("Error", "Files already in OMNI folder")
        return
    elif self.files_in_folder('I') != 0:
        messagebox.showerror("Error", "Files already in IMED folder")
        return

    # Initialize counters
    self.current_run = 0
    self.omni_run_count.set(0)
    self.imed_run_count.set(0)

    # Start the first OMNI stage
    self.omni_running.set(True)
    self.disable_buttons()
    self.monitoring_folders(5, 'O')


In [ ]:
self.total_runs = 5  # total number of OMNI→IMED sequences
self.current_run = 0
